In [9]:
#importando bibliotecas
import pandas as pd
import sqlite3

In [10]:
#conectando o arquivo 
conn = sqlite3.connect('big_data.db') #Lugar onde serão guardados os dados

In [11]:
csv_file  = 'harddrive.csv'  
chunk_size = 100000  #definimos o numero para que a leitura do arquivo seja de 100000 linha por vez

print("isso pode levar alguns minutos")

try:
        for i, chunk in enumerate(pd.read_csv(csv_file, chunksize=chunk_size)):
            mode = 'replace' if i == 0 else 'append'
            chunk.to_sql('monitoramento_discos', conn, if_exists = mode, index= False)
            if(i+1)%10 ==0:
                print(f"{(i+1)*chunk_size}linhas processadas...")
        print("ingestão concluida! O banco está pronto")
except FileNotFoundError: #Caso ele não encontre os dados
    print(f"Erro:Certifique-se de que o arquivo '{csv_file}'está na pasta do seu notebook ")

isso pode levar alguns minutos
1000000linhas processadas...
2000000linhas processadas...
3000000linhas processadas...
ingestão concluida! O banco está pronto


In [12]:
#Se esses dados fossem pequenos, poderiamos fazer simplesmente:

#conn = sqlite3.connect('small_data.db')

#df_pequeno = pd.read_csv('harddrive.csv') 

#conn = sqlite3.connect('small_data.db')


#df_pequeno.to_sql('monitoramento_discos', conn, if_exists='replace', index=False)

#print("Dados carregados instantaneamente!")


#conn.close()

#Pois não teria um esforço extremo da memoria RAM. Como os dados são grandes,particionamos eles

In [13]:
#validação

df_sample = pd.read_sql_query("""
SELECT * FROM monitoramento_discos LIMIT 5
""" , conn)

display(df_sample)

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2016-01-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,135.0,108.0,143,...,None,None,None,None,None,None,None,None,None,None
1,2016-01-01,Z305B2QN,ST4000DM000,1.976651e-311,0,113,54551400,NaN,NaN,96,...,None,None,None,None,None,None,None,None,None,None
2,2016-01-01,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,136.0,104.0,124,...,None,None,None,None,None,None,None,None,None,None
3,2016-01-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,136.0,104.0,137,...,None,None,None,None,None,None,None,None,None,None
4,2016-01-01,WD-WMC4N2899475,WDC WD30EFRX,1.482490e-311,0,200,0,NaN,NaN,175,...,None,None,None,None,None,None,None,None,None,None


In [14]:
#quais os 5 modelos de disco que mais falharam em numeros absolutos?

query_analise = """
SELECT
    model,
    SUM(failure) AS total_falhas,
    COUNT(*) AS total_registros
FROM monitoramento_discos
GROUP by model
HAVING total_falhas > 0 
ORDER BY total_falhas DESC
LIMIT 5 
"""
df_ranking = pd.read_sql_query(query_analise,conn)
print("top 5 modelos com mais falhas:")
display(df_ranking)
    

top 5 modelos com mais falhas:


,model,total_falhas,total_registros
0,ST4000DM000,139,1681473
1,ST320LT007,15,2687
2,Hitachi HDS722020ALA330,13,224052
3,WDC WD800AAJS,6,708
4,WDC WD30EFRX,6,54686


Vamos testar qual modelo é mais estável baseado nas seus registros e falhas:
$$\text{Taxa \%} = \frac{\text{Total de Falhas}}{\text{Total de Registros}} \times 100$$

In [16]:
query_percentual = """
SELECT 
    model, 
    SUM(failure) AS total_falhas,
    COUNT(*) AS total_registros,
    ROUND((SUM(failure) * 100.0) / COUNT(*), 4) AS taxa_falha_percentual
FROM monitoramento_discos
GROUP BY model
HAVING total_registros > 5000 -- Filtro para evitar modelos com poucos testes
ORDER BY taxa_falha_percentual DESC
LIMIT 10;
"""

df_confiabilidade = pd.read_sql_query(query_percentual, conn)
print("🏆 Ranking de Menor estabilidade (Maior Taxa de Erro):")
display(df_confiabilidade)

🏆 Ranking de Menor estabilidade (Maior Taxa de Erro):


,model,total_falhas,total_registros,taxa_falha_percentual
0,WDC WD20EFRX,3,6871,0.0437
1,WDC WD1600AAJS,2,5003,0.0400
2,ST4000DX000,2,10847,0.0184
3,WDC WD30EFRX,6,54686,0.0110
4,ST4000DM000,139,1681473,0.0083
5,Hitachi HDS722020ALA330,13,224052,0.0058
6,WDC WD60EFRX,1,23816,0.0042
7,Hitachi HDS5C4040ALE630,4,136969,0.0029
8,Hitachi HDS5C3030ALA630,5,236690,0.0021
9,ST6000DX000,2,97864,0.0020


Dessa forma decobrimos a falha percentual de cada modelo. Quanto mais abaixo mais estável é o modelo. 

In [19]:
# Nova célula no Jupyter
query_fabricantes = """
SELECT 
    CASE 
        WHEN model LIKE 'ST%' THEN 'Seagate'
        WHEN model LIKE 'WDC%' THEN 'Western Digital'
        WHEN model LIKE 'Hitachi%' THEN 'Hitachi'
        ELSE 'Outros'
    END AS fabricante,
    SUM(failure) AS total_falhas,
    COUNT(*) AS total_amostras,
    ROUND((SUM(failure) * 100.0) / COUNT(*), 4) AS taxa_estabilidade
FROM monitoramento_discos
GROUP BY fabricante
ORDER BY taxa_estabilidade ASC;
"""

df_marcas = pd.read_sql_query(query_fabricantes, conn)
print("📊 Resumo de Estabilidade por Fabricante:")
display(df_marcas)

📊 Resumo de Estabilidade por Fabricante:


,fabricante,total_falhas,total_amostras,taxa_estabilidade
0,Outros,5,575295,0.0009
1,Hitachi,22,650734,0.0034
2,Seagate,162,1839553,0.0088
3,Western Digital,26,113713,0.0229
